In [ ]:
# Manage (global) imports
import networkx as nx
from pyvis.network import Network
from pathlib import Path
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
from random import seed, sample
from json import load

# Manage constants
data_folder = Path() / "data"

In [ ]:
# Load the coordinates for the airports from precomputed data
with open(data_folder / "coordinates.json", "r") as coords_file:
    coords_dict = load(coords_file)
    for k,v in coords_dict.items():
        # Scale and transform the coordinates (improves readability)
        coords_dict[k] = v[0] * 150, v[1] * -150

## Lecture N1

#### Assignment 1: Display your network using pyvis

In [ ]:
# Settings/constants for the visualization
color_map = defaultdict(lambda: "grey", {
    "IT":"green",
    "GB":"blue",
    "FR":"white",
    "NL":"orange",
    "DE":"red",
    "ES":"yellow",
    "NO":"red",
    "BE":"red",
    "IE":"lime",
    "PT":"green",
    "SE":"cyan",
    "DK":"white",
    "PL":"white",
    "GR":"lightblue",
    "CZ":"red",
    "RO":"yellow",
    "HU":"green",
    "AT":"white",
    "CH":"gold",
    "FI":"lightblue",
    "LV":"red",
    "LT":"yellow",
    "EE":"blue",
    "BG":"white",
    "RS":"blue",
    "MT":"gold",
    "HR":"red",
    "SI":"blue",
    "LU":"cyan",
    "AL":"red",
    "BA":"gold",
    "MD":"blue",
    "GI":"blue",
    "SK":"blue",
    "JE":"lightblue",
    "IM":"white"
})
international_color = "rgba(200, 150, 175, 0.15)"

In [ ]:
### This script will display the network in its entirety. 
### If you want you can enable physics for drawing the network
### and display the physics buttons to find a good visualization.

### By default the full network of the airlines is displayed. If you want to
### only see the network of a specific air_line, change air_line_network.gml
### in the next line by one of the following:
### Air_France.gml
### Altalia.gml
### British_Airways.gml
### Easyjet.gml
### KLM.gml
### Lufthansa.gml
### Ryanair.gml
### Scandinavian_Airlines.gml

gml_file = data_folder / "air_line_network.gml"

### Read the gml file as a network
air_line_graph = nx.read_gml(gml_file)


### Create a pyvis network from G
air_line_network = Network("90%", width="100%", bgcolor="#020202", font_color="white", \
                        heading="EU Airline Network")
air_line_network.from_nx(air_line_graph)

### Manipulate node attributes for display purposes
for node in air_line_network.nodes:
    # We first set the size of the node to be equal to a fixed size plus its degree/2
    node["size"] = 3 + int(air_line_graph.degree[node["label"]])/2
    
    ### We set the label of the node to the city name. Then we set the 
    ### title to be the name of the city in bold and include the ICAO and country code
    node["label"] = node["city"]
    node["title"] = "<b>" + node["city"] + "</b> <br>" + node['ICAO'] + ", " + node["country"]  

    ### We set the color of the node based on the country
    node["color"] = color_map[node["country"]]
    ### We set the position of the node based on the coordinates
    node["x"], node["y"] = coords_dict[node["ICAO"]]

### Manipulate edge attributes for display purposes
for edge in air_line_network.edges:
    source, target = air_line_graph.nodes[edge["from"]], air_line_graph.nodes[edge["to"]]

    ### Domestic flights are the same color for the country
    if source["country"] == target["country"]:
        edge["color"] = color_map[source["country"]]
    ### International colors are defined by a constant
    else:
        edge["color"] = international_color

air_line_network.toggle_physics(False)
air_line_network.show("air_line_network.html")

#### Assignment 2: Compute the following characteristics of your network

In [ ]:
gml_file = data_folder / "air_line_network.gml"

### Read the gml file as a network
G = nx.read_gml(gml_file)

#Compute the following characteristics of the network
#1: Average degree
degrees = [d for _,d in G.degree]
avg_degree = sum(degrees) / len(degrees)
print("Average Degree:", avg_degree)

#2: Average clustering
print("Average Clustering:", nx.average_clustering(G))

#3: Average distance
print("Average Distance:", nx.average_shortest_path_length(G))

#4: Diameter
print("Diameter:", nx.diameter(G))

## Lecture N2

#### Assignment 3: Compute the degree distribution of your network and plot it. <br>In addition, compute the second and third moment of your degree distribution

In [ ]:
gml_file = data_folder / "air_line_network.gml"

### Read the gml file as a network
G = nx.read_gml(gml_file)

# Degree distribution
deg_dist = nx.degree_histogram(G)

# Degree histogram plot
plt.title("Degree distribution of the network")
plt.ylabel("No. Nodes")
plt.xlabel("Degree")
plt.bar(range(0, len(deg_dist)), deg_dist)

# General moment function
def nth_moment(g,n):
    s = 0
    for node in g.nodes:
        s += g.degree[node] ** n
    return (s/len(g))

# Second and third moment
print("Second Moment:",nth_moment(G, 2))
print("Third Moment:",nth_moment(G, 3))

#### Assignment 4: Select 100 nodes at random, compute their distance and plot the histogram of the results

In [ ]:
gml_file = data_folder / "air_line_network.gml"

### Read the gml file as a network
G = nx.read_gml(gml_file)

# Select 100 nodes at random
seed(42)
nodes = sample(sorted(G), 100)
RG = G.subgraph(nodes)

# Compute the distances
paths = nx.all_pairs_shortest_path_length(RG)
cntr = Counter()
for _, dists in paths:
    cntr += Counter([d for _, d in dists.items()])
distances = dict(cntr).values()

# Plot the distance histogram
plt.title("Distance distribution of the sampled network")
plt.ylabel("No. Unique Paths")
plt.xlabel("No. Edges on Path")
plt.bar(range(0, len(distances)), distances)
print(end="")

## Lecture N3

#### Assignment 5: No code required, may be added to back the answer if necessary

#### Assignment 6: Use the configuration model to create a random network with the same degree sequence as your network.<br> Compute the four characteristics

In [ ]:
gml_file = data_folder / "air_line_network.gml"

### Read the gml file as a network
G = nx.read_gml(gml_file)

# Generate the configuration model (and drop parallel edges, since this messes with some computations)
CG = nx.Graph(nx.configuration_model([d for _, d in G.degree()], seed=42))

#Compute the four characteristics of the network
#1: Average degree
degrees = [d for _,d in CG.degree]
avg_degree = sum(degrees) / len(degrees)
print("Average Degree:", avg_degree)

#2: Average clustering
print("Average Clustering:", nx.average_clustering(CG))

#3: Average distance
if nx.number_connected_components(CG) == 1:
    print("Average Distance:", nx.average_shortest_path_length(CG))
else:
    for i, c in enumerate(nx.connected_components(CG)):
        print(f"Average Distance for connected subgraph {i+1} of size {len(c)}:", nx.average_shortest_path_length(CG.subgraph(c)))


#4: Diameter
if nx.number_connected_components(CG) == 1:
    print("Diameter:", nx.diameter(CG))
else:
    for i, c in enumerate(nx.connected_components(CG)):
        print(f"Diameter for connected subgraph {i+1} of size {len(c)}:", nx.diameter(CG.subgraph(c)))


## Lecture N4

#### Assignment 7: Compute the epidemic threshold of your network for the SIS and SIR infection models

In [ ]:
gml_file = data_folder / "air_line_network.gml"

### Read the gml file as a network
G = nx.read_gml(gml_file)

# General moment function
def nth_moment(g,n):
    s = 0
    for node in g.nodes:
        s += g.degree[node] ** n
    return (s/len(g))

# Compute required values
degrees = [d for _,d in CG.degree]
k = sum(degrees) / len(degrees)
k2 = nth_moment(G,2)

# SIS
print("Epidemic Threshold SIS:", k/k2)

# SIR
print("Epidemic Threshold SIR:", k/(k2 - k))

#### Assignment 8: Select 5 different nodes at random in your network.<br>For each of these nodes, run 5 complex contagions with different values of α.<br>For each case, document whether there is a complete cascade or else, how large the number of infected nodes is.<br>Visualize the outcome

In [ ]:
# Complex contagion function from the tutorials, used in next cell
def complex_contagion(G,alpha,starting_nodes):
    for node in G.nodes():
        G.nodes[node]['infected'] = 0
    
    infected_nodes = set()
    for node in starting_nodes:
        G.nodes[node]['infected'] = 1
        infected_nodes.add(node)

    infected_nodes_stages = []

    N = G.number_of_nodes()

    num_new_infections = len(infected_nodes)
    total_infections = num_new_infections

    infection_iteration = 0
    
    while (total_infections < N) and (num_new_infections > 0):
        num_new_infections = 0
        new_infected_nodes = set()
        infection_iteration += 1

        for node in infected_nodes:
            ### Checking all the neighbors of the infected node
            for neighbor in G.adj[node]:
                
                ### If the neighbor is not yet infected we check its
                ### neighbors to see if we need to update the infected
                ### status
                if G.nodes[neighbor]['infected'] == 0:
                    
                    number_infected_neighbors = 0
                    degree = G.degree[neighbor]
                    
                    ### Checking infection status of all neighbors of 
                    ### the node (neighbor)
                    for second_neighbor in G.adj[neighbor]:
                        
                        if G.nodes[second_neighbor]['infected'] == 1:
                            number_infected_neighbors += 1
                    
                    ### Check if the fraction of infected neighbors is
                    ### larger than alpha. If yes then we update the 
                    ### infected status of the node
                    if number_infected_neighbors/degree >= alpha:
                        G.nodes[neighbor]['infected'] = 1
                        new_infected_nodes.add(neighbor)
                        num_new_infections += 1

        ### If during this round new nodes were infected we add them to 
        ### the total list of infected nodes. We also add the list of
        ### nodes infected in this round to the list of infected nodes stages
        if (len(new_infected_nodes) > 0):
            
            infected_nodes_stages.append(new_infected_nodes)
            
            for new_node in new_infected_nodes:
                infected_nodes.add(new_node)

    return infected_nodes, infected_nodes_stages

In [ ]:
# Settings for the contagion visualization
start_color = "white"
end_color = "blue"
def_color = "grey"
def_edge_color = "rgba(50,50,50,0.3)"

def getColorFade(c1, c2, s):
    c1=np.array(colors.to_rgb(c1))
    c2=np.array(colors.to_rgb(c2))
    return [colors.to_hex((1-(mix/s))*c1 + (mix/s)*c2) for mix in range(0, s)]

def plot_contagion(G, start_nodes, stages, icao, alpha):
    gradient = getColorFade(start_color, end_color, len(stages) + 1)

    ### Create a pyvis network from G
    network = Network("90%", width="100%", bgcolor="#020202", font_color="white", \
                            heading=f"Contagion from {icao} with alpha {alpha}")
    network.from_nx(G)

    for node in network.nodes:
        # We compute the stage at which the node is infected
        # From this we also derive color (and shape)
        id = node["id"]
        if id in start_nodes:
            stage = "0"
            node["color"] = gradient[0]
            node["shape"] = "diamond"
        else:
            stage = "-"
            node["color"] = def_color
            for i, stage_set in enumerate(stages):
                if id in stage_set:
                    stage = str(i + 1)
                    node["color"] = gradient[i + 1]
                    break
        G.nodes[id]["stage"] = stage

        # We set the size of the node to be equal to a fixed size plus its degree/2
        node["size"] = 3 + int(G.degree[node["label"]])/2

        ### We set the label of the node to the city name and the infection stage. Then we set the 
        ### title to be the name of the city and the infection stage in bold and include the ICAO and country code
        node["label"] = node["city"] + " (" + stage + ")"
        node["title"] = "<b>" + node["city"] + " (" + stage + ")" + "</b> <br>" + node['ICAO'] + ", " + node["country"] + "<br>Degree: " + str(G.degree(id))

        ### We set the position of the node based on the coordinates
        node["x"], node["y"] = coords_dict[node["ICAO"]]

    for edge in network.edges:
        source, target = G.nodes[edge["from"]], G.nodes[edge["to"]]
        
        if source["stage"] != "-" and target["stage"] != "-":
            ### Edges that helped contagions
            ss, ts = int(source["stage"]), int(target["stage"])
            edge["color"] = gradient[max(ss, ts)]
        else:
            ### Edges that did not help contagions
            edge["color"] = def_edge_color


    network.toggle_physics(False)
    network.write_html("./visualizations/" + icao + "_" + str(alpha) + ".html")

In [ ]:
gml_file = data_folder / "air_line_network.gml"

### Read the gml file as a network
G = nx.read_gml(gml_file)

# Select 5 different start nodes at random
seed(42)
random_start_nodes = sample(sorted(G), 5)

# Pick 5 different alpha values
alphas = [0.05, 0.1, 0.2, 0.5, 0.75]

# For each start node and alpha, simulate the contagion
for start_node in random_start_nodes:
    city, icao = G.nodes('city')[start_node], G.nodes('ICAO')[start_node]
    print(f"Start node {start_node} representing {city} ({icao}):")
    for alpha in alphas:    
        ### Compute and report the results
        infected, stages = complex_contagion(G, alpha, [start_node])
        print(f"Alpha: {alpha}, Total infections: {len(infected)}, No. stages: {len(stages)}")

        ### Write the html containing the visualization
        plot_contagion(G, [start_node], stages, icao, alpha)
    print()


#### Assignment 9: No code required

## Lecture N5

#### Assignment 10: Find the four nodes in your network with the highest centrality scores

In [ ]:
gml_file = data_folder / "air_line_network.gml"

### Read the gml file as a network
G = nx.read_gml(gml_file)

# Closeness centrality score
cc = nx.closeness_centrality(G)
max_cc = max(cc, key=cc.get)

# Harmonic centrality score
hc = nx.harmonic_centrality(G)
max_hc = max(hc, key=hc.get)

# Betweenness centrality score
bc = nx.betweenness_centrality(G)
max_bc = max(bc, key=bc.get)

# PageRank centrality score
pc = nx.pagerank(G)
max_pc = max(pc, key=pc.get)

# Results
getNodeInfo = lambda G,n: " - ".join(G.nodes[n].values())

print("Highest Closeness Centrality:  ", getNodeInfo(G, max_cc))
print("Highest Harmonic Centrality:   ", getNodeInfo(G, max_hc))
print("Highest Betweenness Centrality:", getNodeInfo(G, max_bc))
print("Highest PageRank Centrality:   ", getNodeInfo(G, max_pc))

#### Assignment 11: Visualize your network and find each of the four nodes from the previous assignment

In [ ]:
node_color = "grey"
cc_color = "lime"
hc_color = "purple"
bc_color = "yellow"
pc_color = "red"
edge_color = "rgba(50,50,50,0.9)"

gml_file = data_folder / "air_line_network.gml"

### Read the gml file as a network
G = nx.read_gml(gml_file)

### Create a pyvis network from G
network = Network("90%", width="100%", bgcolor="#020202", font_color="white", \
                        heading="Highest Centrality")
network.from_nx(air_line_graph)

### Manipulate node attributes for display purposes
for node in network.nodes:
    # We first set the size of the node to be equal to a fixed size plus its degree/3
    node["size"] = 3 + int(air_line_graph.degree[node["label"]])/2
    
    ### We set the label of the node to the city name. Then we set the 
    ### title to be the name of the city in bold and include the ICAO and country code
    node["label"] = node["city"]
    node["title"] = "<b>" + node["city"] + "</b> <br>" + node['ICAO'] + ", " + node["country"]  

    ### We set the color of the node based on the country
    if node["id"] == max_cc:
        node["shape"] = "diamond"
        node["color"] = cc_color
    elif node["id"] == max_hc:
        node["shape"] = "diamond"
        node["color"] = hc_color
    elif node["id"] == max_bc:
        node["shape"] = "diamond"
        node["color"] = bc_color
    elif node["id"] == max_pc:
        node["shape"] = "diamond"
        node["color"] = pc_color
    else:
        node["color"] = node_color
    ### We set the position of the node based on the coordinates
    node["x"], node["y"] = coords_dict[node["ICAO"]]

### Manipulate edge attributes for display purposes
for edge in network.edges:
    edge["color"] = edge_color

### Comment the next line to enable the physics for drawing the graph
network.toggle_physics(False)

network.show("highest_centrality.html")

#### Assignment 12: No code required